In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
from requests_toolbelt.multipart.decoder import MultipartDecoder
import sys


In [2]:
import logging
logging.basicConfig(level=logging.INFO)
pd.options.display.max_colwidth = 300

In [3]:
connectId = "228b427a-fded-4f00-9602-1225879e86a6"
wcs_path = 'https://securewatch.maxar.com/deliveryservice/wcsaccess'
order = './order.csv'

In [4]:
from pathlib import Path
Path('./metadata').mkdir(exist_ok=True)
Path('./images').mkdir(exist_ok=True)

In [5]:
order_df = pd.read_csv('./order.csv')
print("order size", len(order_df))

order size 188


In [ ]:
%%time

debug = False

fmt, ext = ('image/jpeg', '.jpeg') if debug else ('image/geotiff', '.tiff')
order = order_df[4:5].iterrows() if debug else order_df.iterrows()

for (index, row) in tqdm(order):
    image_id = row['featureId']
    bbox = row['bbox']
    basename = row['uid']

    with open("./images/" + basename + '-metadata.json', 'w') as f:
        f.write(row.to_json())
        
    payload={
        "connectId": connectId,
        "Service": "WCS",
        "Request": "GetCoverage",
        "version": "1.1.0",
        "srsName": "EPSG:4326",
        "typeName": "FinishedFeature",
        "WIDTH": "1",
        "HEIGHT": "1",
        "format": fmt,
        "Identifier": image_id,
        "boundingBox": bbox
    }

    try: 
        logging.info(f"Fetching {image_id} for {basename}")
        resp = requests.get(wcs_path, payload)
        # resp.raise_for_status()
        logging.debug(str(resp.request))
        decoder = MultipartDecoder.from_response(resp)
        parts = decoder.parts
    except Exception as e:
        logger.error(e)
        with open("./images/" + basename + "-error.log", 'w') as f:
            tpe, value, traceback = sys.exc_info()
            f.write(f"{str(tpe)}\n{str(value)}\n{str(traceback)}")
        continue

    with open("./images/" + basename + ext, 'wb') as f:
        f.write(parts[1].content)